In [ ]:
!pip show roboflow ultralytics


In [3]:
from roboflow import Roboflow
import os
from ultralytics import YOLO
from datetime import datetime
import cv2
import numpy as np
from pathlib import Path

## Descarregar el dataset

In [ ]:
rf = Roboflow(api_key="5HYzrdPm6LO8xmXDVn3G")
project = rf.workspace("vc").project("matriculas-espanolas")
version = project.version(1)
dataset = version.download("yolov11")                

## Entrenar el model

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)

## Guardar el model

In [ ]:
models_path = "models"
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
model_name = f"model_{fecha_hora_actual}"
model_path = os.path.join(models_path, model_name)
os.makedirs(models_path, exist_ok=True)
model.save(f"{model_path}.pt")

## Carregar el model

In [ ]:
# Configuración
model_path = "./models/model_27_03_2025_19_30.pt"
input_path = "./Matriculas-Españolas-1/valid/images"
output_dir = "./predicted"
os.makedirs(output_dir, exist_ok=True)

# Cargar modelo
model_matr = YOLO(model_path)

## Prediure imagues

In [ ]:
def process_input(input_path: str | list[str]) -> list[str]:
    """Convierte el input en una lista de rutas de imágenes válidas"""
    if isinstance(input_path, str):
        if os.path.isdir(input_path):
            # Obtener todas las imágenes de la carpeta
            return [
                os.path.join(input_path, f) 
                for f in os.listdir(input_path) 
                if f.lower().endswith(('.png', '.jpg', '.jpeg'))
            ]
        return [input_path]  # Es una sola imagen
    elif isinstance(input_path, list):
        return input_path  # Ya es una lista de imágenes
    raise ValueError("El input debe ser: 1 imagen, lista de imágenes o carpeta")

def initialize_markdown(model_path: str) -> str:
    """Crea el contenido inicial del markdown"""
    return (
        f"# Resultados de predicción\n\n"
        f"Modelo usado: `{model_path}`\n"
        f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    )

def process_image(
    model, 
    image_path: str, 
    output_dir: str,
    confidence_threshold: float = 0.25
) -> tuple:
    """
    Procesa una imagen y devuelve:
    - imagen con bboxes dibujados
    - lista de detecciones
    - ruta de salida
    """
    # Realizar predicción
    results = model.predict(
        source=image_path,
        conf=confidence_threshold,
        save=False,
        verbose=False
    )
    
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    
    detections = []
    result = results[0]
    
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = float(box.conf[0])
        class_id = int(box.cls[0])
        class_name = result.names[class_id]
        
        # Dibujar bounding box
        color = (0, 255, 0)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        
        # Añadir etiqueta
        label = f"{class_name}: {confidence:.2f}"
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - text_height - 5), (x1 + text_width, y1), color, -1)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        detections.append({
            "class": class_name,
            "confidence": confidence,
            "bbox": [x1, y1, x2, y2]
        })
    
    output_path = os.path.join(output_dir, f"pred_{Path(image_path).name}")
    return image, detections, output_path

def update_markdown(
    markdown_content: str,
    image_path: str,
    output_path: str,
    detections: list,
    image_index: int
) -> str:
    """Actualiza el contenido markdown con los nuevos resultados"""
    markdown_content += f"## Imagen {image_index}: {Path(image_path).name}\n"
    markdown_content += f"![Imagen procesada](../{output_path})\n\n"
    markdown_content += "**Detecciones:**\n"
    
    for det in detections:
        markdown_content += (
            f"- Clase: `{det['class']}` | "
            f"Confianza: `{det['confidence']:.2f}` | "
            f"BBox: `{det['bbox']}`\n"
        )
    
    return markdown_content + "\n---\n\n"

def save_results(
    markdown_content: str,
    output_dir: str,
    output_path: str,
    processed_image: np.ndarray
):
    """Guarda todos los resultados"""
    # Guardar imagen procesada
    cv2.imwrite(output_path, processed_image)
    
    # Guardar markdown
    with open(os.path.join(output_dir, "results.md"), "w", encoding="utf-8") as f:
        f.write(markdown_content)

def main(
    model_path: str,
    input_path: str | list[str],
    output_dir: str = "./predicted",
    confidence_threshold: float = 0.25
):
    """Flujo principal de ejecución"""
    # Configuración inicial
    os.makedirs(output_dir, exist_ok=True)
    model = YOLO(model_path)
    
    # Procesar input
    images = process_input(input_path)
    
    # Inicializar markdown
    markdown_content = initialize_markdown(model_path)
    
    # Procesar cada imagen
    for i, image_path in enumerate(images, 1):
        try:
            processed_img, detections, out_path = process_image(
                model, image_path, output_dir, confidence_threshold
            )
            
            markdown_content = update_markdown(
                markdown_content, image_path, out_path, detections, i
            )
            
            save_results(
                markdown_content, output_dir, out_path, processed_img
            )
            
        except Exception as e:
            print(f"Error procesando {image_path}: {str(e)}")
            continue
    
    print(f"Procesamiento completado. Resultados en: {output_dir}")

if __name__ == "__main__":
    # Ejemplo de uso
    main(
        model_path="best.pt",
        input_path="./Matriculas-Españolas-1/valid/images",
        confidence_threshold=0.25
    )

Procesamiento completado. Resultados guardados en: ./predicted
